In [8]:
import datetime
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd
from datetime import datetime

In [10]:
full_demand = pd.read_csv('./raw/household_power_consumption.txt', sep=';', low_memory=False)

In [11]:
full_demand

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0
...,...,...,...,...,...,...,...,...,...
2075254,26/11/2010,20:58:00,0.946,0.000,240.430,4.000,0.000,0.000,0.0
2075255,26/11/2010,20:59:00,0.944,0.000,240.000,4.000,0.000,0.000,0.0
2075256,26/11/2010,21:00:00,0.938,0.000,239.820,3.800,0.000,0.000,0.0
2075257,26/11/2010,21:01:00,0.934,0.000,239.700,3.800,0.000,0.000,0.0


In [12]:
demand = full_demand[['Date', 'Time', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].copy()
demand.loc[:,'Datetime'] = pd.to_datetime(demand.loc[:, 'Date'] + ' ' + demand.loc[:,'Time'], dayfirst=True)
demand.set_index('Datetime', inplace=True)
demand = demand.loc[:, ['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]

In [13]:
demand

,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,
2006-12-16 17:24:00,0.000,1.000,17.0
2006-12-16 17:25:00,0.000,1.000,16.0
2006-12-16 17:26:00,0.000,2.000,17.0
2006-12-16 17:27:00,0.000,1.000,17.0
2006-12-16 17:28:00,0.000,1.000,17.0
...,...,...,...
2010-11-26 20:58:00,0.000,0.000,0.0
2010-11-26 20:59:00,0.000,0.000,0.0
2010-11-26 21:00:00,0.000,0.000,0.0


In [15]:
demand_episodes = demand.groupby(pd.Grouper(freq='W'))

In [16]:
demand_episodes.describe()

Sub_metering_3                                                
                    count      mean       std  min  25%   50%   75%   max
Datetime                                                                 
2006-12-17         1836.0  9.949346  8.501908  0.0  0.0  17.0  17.0  19.0
2006-12-24        10078.0  7.684759  8.709914  0.0  0.0   0.0  17.0  20.0
2006-12-31        10078.0  6.671661  8.534913  0.0  0.0   0.0  17.0  20.0
2007-01-07        10080.0  5.376290  8.074240  0.0  0.0   0.0  17.0  19.0
2007-01-14        10079.0  8.270265  8.694638  0.0  0.0   0.0  17.0  19.0
...                   ...       ...       ...  ...  ...   ...   ...   ...
2010-10-31        10080.0  5.346825  8.082946  0.0  1.0   1.0  11.0  31.0
2010-11-07        10080.0  6.021925  8.276141  0.0  1.0   1.0  17.0  31.0
2010-11-14        10080.0  7.740476  8.620190  0.0  0.0   1.0  17.0  29.0
2010-11-21        10080.0  6.285317  8.423491  0.0  0.0   0.0  17.0  20.0
2010-11-28         7023.0  6.412075  8.418418  0.0  0.0   0.0  17.0  19.0

[207 rows x 8 columns]

In [19]:
demand_episodes = pd.concat([group.fillna(0) for name, group in demand_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2007, 1, 1) and group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))

In [20]:
demand_episodes.describe()

Sub_metering_3                                                
                    count      mean       std  min  25%   50%   75%   max
Datetime                                                                 
2007-01-07        10080.0  5.376290  8.074240  0.0  0.0   0.0  17.0  19.0
2007-01-14        10080.0  8.269444  8.694597  0.0  0.0   0.0  17.0  19.0
2007-01-21        10080.0  8.086508  8.714090  0.0  0.0   0.0  17.0  19.0
2007-01-28        10080.0  7.149008  8.681189  0.0  0.0   0.0  18.0  20.0
2007-02-04        10080.0  9.448810  8.704572  0.0  0.0  17.0  18.0  19.0
...                   ...       ...       ...  ...  ...   ...   ...   ...
2008-11-30        10080.0  8.086508  8.820860  0.0  0.0   0.0  18.0  19.0
2008-12-07        10080.0  7.327480  8.773557  0.0  0.0   0.0  18.0  20.0
2008-12-14        10080.0  7.127976  8.775631  0.0  0.0   0.0  18.0  21.0
2008-12-21        10080.0  8.186111  8.891151  0.0  0.0   0.0  18.0  20.0
2008-12-28        10080.0  5.717659  8.364338  0.0  0.0   0.0  18.0  20.0

[104 rows x 8 columns]

In [23]:
with pd.HDFStore('./minutely/demand.h5') as store:
    # Save each DataFrame with a key
    i = 0
    for name, group in demand_episodes:
        if not group.empty:
            store[f'eps_{i}'] = group
            i += 1